In [11]:
import quandl
quandl.ApiConfig.api_key = '1w1X-kbMsxdg4Ts1disD'

In [50]:
import numpy as np
import random
import pandas as pd
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import matplotlib.pyplot as plt
import seaborn as sns
import requests

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from datetime import datetime, date, timedelta

from pandas.tseries.offsets import MonthEnd

In [3]:
tickers = ['SPY','VUG','VTV','MTUM','QUAL',
           'DVY',
           'VB',
           'VWO',
           'TLT','GLD','SHV',
           'XLK','XLE','XLF',
           'ARKK','GBTC','VIXY'
          ]
factors = ['S&P 500','Growth','Value','Momentum','Quality',
           'Dividends',
           'Small Cap',
           'Emerging Markets',
           'Treasury Bonds','Gold','Cash',
           'Technology','Energy','Financials',
           'ARK Innovation ETF','Bitcoin','Vix Futures'
          ]
fund_df = quandl.get_table('SHARADAR/SFP', ticker=tickers, paginate=True)
fund_df.sort_values(by='date', ascending=True, inplace=True)

In [7]:
master = fund_df[fund_df['ticker']=='SPY'][['date','closeadj']].copy(deep=True)
master.set_index('date', inplace=True)
master.rename({'closeadj': factors[0]}, axis=1, inplace=True)

for i, ticker in enumerate(tickers[1:]):
    merge_df = fund_df[fund_df['ticker']==ticker][['date','closeadj']].set_index('date')
    master = master.merge(merge_df, how='left', left_index=True, right_index=True)
    master.rename({'closeadj': factors[i+1]}, axis=1, inplace=True)
    
master.dropna(inplace=True)

In [28]:
returns_df = master/master.shift(1)-1
returns_df = returns_df.dropna()

In [22]:
def dates_bwn_twodates(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

def get_fred_data(series_id, start, end='9999-12-31'):
    payload = {'api_key': 'b48bfbb0488f76bd608c39b36bad3cf9', 
               'file_type': 'json', 
               'series_id': series_id,
               'observation_start': start,
               'observation_end': end
              }
    response = requests.get('https://api.stlouisfed.org/fred/series/observations', params=payload)
    output = response.json()
    output = output['observations']
    
    payload3 = {'api_key': 'b48bfbb0488f76bd608c39b36bad3cf9', 
               'file_type': 'json', 
               'series_id': series_id
              }
    response3 = requests.get('https://api.stlouisfed.org/fred/series', params=payload3)
    name = response3.json()
    title = name['seriess'][0]['title']
    
    date_list = []
    output_list = []
    index = 0
    for y in output:
        date_list.append(datetime.strptime(y['date'], '%Y-%m-%d'))
        try:
            output_list.append(float(y['value']))
        except:
            try:
                output_list.append(output_list[index-1])
            except:
                output_list.append(None)
        index += 1
    
    output_df = pd.DataFrame()
    output_df['date'] = date_list
    output_df['values'] = output_list
    return output_df, title

In [134]:
macro_tickers = ['VIXCLS', 'INDPRO', 'CPIAUCSL']
macro_labels = ['VIX', 'Indust Prod', 'Inflation']

for idx, i in enumerate(macro_tickers):
    if idx == 0:
        macro_df = get_fred_data(i,'1990-01-01')
        macro_df = macro_df[0].set_index('date')
        macro_df.rename({'values': macro_labels[idx]}, axis=1, inplace=True)
    else:
        temp_df = get_fred_data(i,'1990-01-01')
        temp_df[0]['date'] = temp_df[0]['date'] + MonthEnd(1)
        temp_df = temp_df[0].set_index('date')
        temp_df.rename({'values': macro_labels[idx]}, axis=1, inplace=True)
        macro_df = macro_df.merge(temp_df, how='left', left_index=True, right_index=True)
    
macro_df = macro_df.asfreq('D').ffill().resample('M').asfreq()

In [145]:
all_dates = []
for d in dates_bwn_twodates(final.index[0], final.index[-1]):
    all_dates.append(d)

master_returns_df = pd.DataFrame(all_dates, columns=['date'])
master_returns_df = master_returns_df.merge(returns_df, how='left', left_on='date', right_index=True)
master_returns_df.set_index('date', inplace=True)
master_returns_df = master_returns_df.fillna(0)
cumprod_df = (1+master_returns_df).cumprod()

master_returns_df = macro_df.merge(cumprod_df, how='left', left_index=True, right_index=True)
master_returns_df = master_returns_df/master_returns_df.shift(1) - 1
master_returns_df['Inflation Delta'] = master_returns_df['Inflation'] - master_returns_df['Inflation'].shift(1)
master_returns_df = master_returns_df.dropna()

master_returns_df['VIX Rising'] = [True if i>0 else False for i in master_returns_df['VIX']]
master_returns_df['IndPro Rising'] = [True if i>0 else False for i in master_returns_df['Indust Prod']]
master_returns_df['Inflation Accelerating'] = [True if i>0 else False for i in master_returns_df['Inflation Delta']]

In [156]:
# Quad analysis
# Box 1 = IndPro down, Inflation decelerating
# Box 2 = IndPro down, Inflation accelrating
# Box 3 = IndPro up, Inflation decelerating
# Box 4 = IndPro up, Inflation accelrating

pd.options.display.float_format = '{:.2%}'.format

var1 = 'Indust Prod'
var2 = 'Inflation Delta'

box = []
for i in master_returns_df.iterrows():
    row = i[1]
    if (row[var1] < 0) & (row[var2] < 0):
        box.append(1)
    elif (row[var1] < 0) & (row[var2] >= 0):
        box.append(2)
    elif (row[var1] >= 0) & (row[var2] < 0):
        box.append(3)
    else:
        box.append(4)
master_returns_df['box'] = box
quads_df = master_returns_df.groupby(by='box').mean()

VIX_df = master_returns_df.groupby(by='VIX Rising').mean()
IndPro_df = master_returns_df.groupby(by='IndPro Rising').mean()
inflation_df = master_returns_df.groupby(by='Inflation Accelerating').mean()

quads_df[['S&P 500','Treasury Bonds','Small Cap','Emerging Markets','Gold','Bitcoin']]

,S&P 500,Treasury Bonds,Small Cap,Emerging Markets,Gold,Bitcoin
box,,,,,,
1,0.00%,0.62%,-0.50%,-2.03%,0.37%,-1.49%
2,0.73%,-0.34%,0.89%,1.22%,0.59%,15.03%
3,0.71%,1.13%,0.42%,-0.22%,0.70%,10.47%
4,2.67%,-0.13%,2.38%,2.50%,0.51%,15.52%


In [157]:
VIX_df[['S&P 500','Treasury Bonds','Small Cap','Emerging Markets','Gold','Bitcoin']]

,S&P 500,Treasury Bonds,Small Cap,Emerging Markets,Gold,Bitcoin
VIX Rising,,,,,,
False,3.30%,0.02%,3.35%,2.27%,0.01%,12.47%
True,-1.00%,0.86%,-1.59%,-1.30%,1.15%,8.72%


In [158]:
IndPro_df[['S&P 500','Treasury Bonds','Small Cap','Emerging Markets','Gold','Bitcoin']]

,S&P 500,Treasury Bonds,Small Cap,Emerging Markets,Gold,Bitcoin
IndPro Rising,,,,,,
False,0.65%,0.49%,0.57%,-0.08%,0.56%,5.03%
True,2.06%,0.33%,1.53%,1.50%,0.58%,19.51%


In [160]:
inflation_df[['S&P 500','Treasury Bonds','Small Cap','Emerging Markets','Gold','Bitcoin']]

,S&P 500,Treasury Bonds,Small Cap,Emerging Markets,Gold,Bitcoin
Inflation Accelerating,,,,,,
False,0.53%,0.81%,0.26%,-0.75%,0.44%,6.98%
True,2.02%,-0.05%,1.78%,2.10%,0.72%,15.11%
